### Imports

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

In [ ]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "rainfall/"

In [3]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files


[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [4]:
%%time
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])


CPU times: user 1.93 s, sys: 2.41 s, total: 4.34 s
Wall time: 43.9 s


In [5]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(os.path.join(output_directory,'data'))

CPU times: user 10.1 s, sys: 1.54 s, total: 11.7 s
Wall time: 12.1 s


In [6]:
%ls -ltr rainfall/data/


total 10448008
-rw-r--r--   1 tanmayagarwal  staff   95376895 31 Mar 17:01 MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
-rw-r--r--   1 tanmayagarwal  staff   94960113 31 Mar 17:01 AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 tanmayagarwal  staff   82474546 31 Mar 17:01 NorESM2-LM_daily_rainfall_NSW.csv
-rw-r--r--   1 tanmayagarwal  staff  127613760 31 Mar 17:01 ACCESS-CM2_daily_rainfall_NSW.csv
-rw-r--r--   1 tanmayagarwal  staff  232118894 31 Mar 17:01 FGOALS-f3-L_daily_rainfall_NSW.csv
-rw-r--r--   1 tanmayagarwal  staff  330360682 31 Mar 17:01 CMCC-CM2-HR4_daily_rainfall_NSW.csv
-rw-r--r--   1 tanmayagarwal  staff  254009247 31 Mar 17:02 MRI-ESM2-0_daily_rainfall_NSW.csv
-rw-r--r--   1 tanmayagarwal  staff  235661418 31 Mar 17:02 GFDL-CM4_daily_rainfall_NSW.csv
-rw-r--r--   1 tanmayagarwal  staff  294260911 31 Mar 17:02 BCC-CSM2-MR_daily_rainfall_NSW.csv
-rw-r--r--   1 tanmayagarwal  staff  295768615 31 Mar 17:02 EC-Earth3-Veg-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 tanmayagarwal

In [7]:
%%time
### just listing to get an idea how individual file looks like 
use_cols = ['time', 'lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)']
df = pd.read_csv("rainfall/data/ACCESS-CM2_daily_rainfall_NSW.csv", usecols=use_cols,dtype={'TailNum': 'str'})
df


CPU times: user 816 ms, sys: 80.5 ms, total: 896 ms
Wall time: 907 ms


,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
0,1889-01-01 12:00:00,-36.25,-35.00,140.625,142.50,3.293256e-13
1,1889-01-02 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
2,1889-01-03 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
3,1889-01-04 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
4,1889-01-05 12:00:00,-36.25,-35.00,140.625,142.50,1.047658e-02
...,...,...,...,...,...,...
1932835,2014-12-27 12:00:00,-30.00,-28.75,151.875,153.75,2.951144e-02
1932836,2014-12-28 12:00:00,-30.00,-28.75,151.875,153.75,2.257118e-01
1932837,2014-12-29 12:00:00,-30.00,-28.75,151.875,153.75,1.204670e-01
1932838,2014-12-30 12:00:00,-30.00,-28.75,151.875,153.75,2.632404e-02


In [8]:
%ls -ltr rainfall/data/


total 10448008
-rw-r--r--   1 tanmayagarwal  staff   95376895 31 Mar 17:01 MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
-rw-r--r--   1 tanmayagarwal  staff   94960113 31 Mar 17:01 AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 tanmayagarwal  staff   82474546 31 Mar 17:01 NorESM2-LM_daily_rainfall_NSW.csv
-rw-r--r--   1 tanmayagarwal  staff  127613760 31 Mar 17:01 ACCESS-CM2_daily_rainfall_NSW.csv
-rw-r--r--   1 tanmayagarwal  staff  232118894 31 Mar 17:01 FGOALS-f3-L_daily_rainfall_NSW.csv
-rw-r--r--   1 tanmayagarwal  staff  330360682 31 Mar 17:01 CMCC-CM2-HR4_daily_rainfall_NSW.csv
-rw-r--r--   1 tanmayagarwal  staff  254009247 31 Mar 17:02 MRI-ESM2-0_daily_rainfall_NSW.csv
-rw-r--r--   1 tanmayagarwal  staff  235661418 31 Mar 17:02 GFDL-CM4_daily_rainfall_NSW.csv
-rw-r--r--   1 tanmayagarwal  staff  294260911 31 Mar 17:02 BCC-CSM2-MR_daily_rainfall_NSW.csv
-rw-r--r--   1 tanmayagarwal  staff  295768615 31 Mar 17:02 EC-Earth3-Veg-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 tanmayagarwal

['rainfall/data/MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv',
 'rainfall/data/AWI-ESM-1-1-LR_daily_rainfall_NSW.csv',
 'rainfall/data/NorESM2-LM_daily_rainfall_NSW.csv',
 'rainfall/data/ACCESS-CM2_daily_rainfall_NSW.csv',
 'rainfall/data/FGOALS-f3-L_daily_rainfall_NSW.csv',
 'rainfall/data/CMCC-CM2-HR4_daily_rainfall_NSW.csv',
 'rainfall/data/MRI-ESM2-0_daily_rainfall_NSW.csv',
 'rainfall/data/GFDL-CM4_daily_rainfall_NSW.csv',
 'rainfall/data/BCC-CSM2-MR_daily_rainfall_NSW.csv',
 'rainfall/data/EC-Earth3-Veg-LR_daily_rainfall_NSW.csv',
 'rainfall/data/CMCC-ESM2_daily_rainfall_NSW.csv',
 'rainfall/data/NESM3_daily_rainfall_NSW.csv',
 'rainfall/data/MPI-ESM1-2-LR_daily_rainfall_NSW.csv',
 'rainfall/data/ACCESS-ESM1-5_daily_rainfall_NSW.csv',
 'rainfall/data/FGOALS-g3_daily_rainfall_NSW.csv',
 'rainfall/data/INM-CM4-8_daily_rainfall_NSW.csv',
 'rainfall/data/MPI-ESM1-2-HR_daily_rainfall_NSW.csv',
 'rainfall/data/TaiESM1_daily_rainfall_NSW.csv',
 'rainfall/data/NorESM2-MM_daily_rainfall_NSW.csv

In [13]:
%%time
## here we are using a normal python way for merging the data 
import pandas as pd
files = glob.glob('rainfall/data/*.csv')
files.remove( 'rainfall/data/observed_daily_rainfall_SYD.csv')
df = pd.concat(pd.read_csv(file, index_col=0, parse_dates=True, 
            usecols=use_cols).assign(model = re.findall("/([^_]*)", file)[0])
          for file in files)
              
df.to_csv("rainfall/combined_data.csv")


CPU times: user 5min 7s, sys: 11.7 s, total: 5min 19s
Wall time: 5min 25s


In [62]:
df

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM
...,...,...,...,...,...,...
2014-12-27 12:00:00,-30.157068,-29.214660,153.1250,154.3750,6.689683e+00,SAM0-UNICON
2014-12-28 12:00:00,-30.157068,-29.214660,153.1250,154.3750,7.862555e+00,SAM0-UNICON
2014-12-29 12:00:00,-30.157068,-29.214660,153.1250,154.3750,1.000503e+01,SAM0-UNICON


In [14]:
%%sh
du -sh rainfall/combined_data.csv


5.9G	rainfall/combined_data.csv


In [15]:
print(df.shape)


(62467843, 6)


In [16]:
df.columns

Index(['lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)', 'model'], dtype='object')

### Trying to load data to memory

In [4]:
df = pd.read_csv("rainfall/combined_data.csv")

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [8]:
print("Memory taken up by combined_data is {} MB".format(df.memory_usage().sum()/1e6))

Memory taken up by combined_data is 3498.199336 MB


#### Using less columns

In [17]:
combined_data = pd.read_csv("rainfall/combined_data.csv", usecols=['time', 'rain (mm/day)','model'])

In [19]:
print("Memory taken up by reduced data is {} MB".format(combined_data.memory_usage().sum()/1e6))

Memory taken up by reduced data is 1499.22836 MB


#### Using dtype to reduce size

In [ ]:
import numpy as np

In [ ]:
dtype_dict = {
    'lat_min': np.float16,
    'lat_max': np.float16,
    'lon_min': np.float16,
    'lon_max': np.float16,
    'rain (mm/day)': np.float32,

}

In [ ]:
df = pd.read_csv("rainfall/combined_data.csv", dtype=dtype_dict)

In [ ]:
print("Memory taken up by manipulating dtypes is {} MB".format(df.memory_usage().sum()/1e6))

### Perform a simple EDA in Python

In [32]:
combined_data.head()

,time,rain (mm/day),model
0,1889-01-01 12:00:00,4.244226e-13,data/MPI-ESM-1-2-HAM
1,1889-01-02 12:00:00,4.217326e-13,data/MPI-ESM-1-2-HAM
2,1889-01-03 12:00:00,4.498125e-13,data/MPI-ESM-1-2-HAM
3,1889-01-04 12:00:00,4.251282e-13,data/MPI-ESM-1-2-HAM
4,1889-01-05 12:00:00,4.270161e-13,data/MPI-ESM-1-2-HAM


In [40]:
%%time
combined_data['time'] = pd.to_datetime(combined_data, infer_datetime_format=True)

In [ ]:
%%time
df = pd.DataFrame(combined_rain.groupby([combined_rain['time'].dt.year, combined_rain['model']])['rain (mm/day)'].mean()).reset_index()


In [ ]:
%%time
year = 2013
fig = px.bar(df[df['time']==year], x="model", y="rain (mm/day)")
fig.show()


In [ ]:
%%time
Model = "MIROC6"
fig = px.bar(df[df['model']==Model], x="time", y="rain (mm/day)")
fig.show()


In [ ]:
%%time
fig = px.scatter(df, x="time", y="rain (mm/day)", color = "model")
fig.show()


### Perform a simple EDA in R


In [2]:
import os
os.environ['R_HOME'] = '/Users/tanmayagarwal/opt/miniconda3/envs/525_dev/lib/R'







In [3]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from IPython.display import display
import pyarrow.dataset as ds
import pyarrow as pa
from pyarrow import csv
import rpy2_arrow.pyarrow_rarrow as pyra

In [4]:
%load_ext rpy2.ipython


In [5]:
%%time
dataset = ds.dataset("rainfall/combined_data.csv", format="csv")
# Converting the `pyarrow dataset` to a `pyarrow table`
table = dataset.to_table()
# Converting a `pyarrow table` to a `rarrow table`
r_table = pyra.converter.py2rpy(table)

CPU times: user 12.9 s, sys: 1.2 s, total: 14.1 s
Wall time: 13.2 s


In [7]:
%%time
%%R -i r_table
start_time <- Sys.time()

suppressMessages(library(dplyr))
result <- r_table |>
    glimpse()
    
print(result)

end_time <- Sys.time()
print(end_time - start_time)


Table
62,467,843 rows x 7 columns
$ time      <timestamp[s]> 1889-01-01 04:00:00, 1889-01-02 04:00:00, 1889-01-03 …
$ lat_min         <double> -35.43987, -35.43987, -35.43987, -35.43987, -35.43987…
$ lat_max         <double> -33.57462, -33.57462, -33.57462, -33.57462, -33.57462…
$ lon_min         <double> 141.5625, 141.5625, 141.5625, 141.5625, 141.5625, 141…
$ lon_max         <double> 143.4375, 143.4375, 143.4375, 143.4375, 143.4375, 143…
$ `rain (mm/day)` <double> 4.244226e-13, 4.217326e-13, 4.498125e-13, 4.251282e-1…
$ model           <string> "data/MPI-ESM-1-2-HAM", "data/MPI-ESM-1-2-HAM", "data…
Table
62467843 rows x 7 columns
$time <timestamp[s]>
$lat_min <double>
$lat_max <double>
$lon_min <double>
$lon_max <double>
$rain (mm/day) <double>
$model <string>
Time difference of 0.157701 secs
CPU times: user 125 ms, sys: 32.9 ms, total: 158 ms
Wall time: 179 ms


In [9]:
%%time
%%R -i r_table
start_time <- Sys.time()

suppressMessages(library(dplyr))
result <- r_table |> count(model)
    
print(result |>  collect())

end_time <- Sys.time()
print(end_time - start_time)


# A tibble: 27 × 2
   model                       n
   <chr>                   <int>
 1 data/MPI-ESM-1-2-HAM   966420
 2 data/AWI-ESM-1-1-LR    966420
 3 data/NorESM2-LM        919800
 4 data/ACCESS-CM2       1932840
 5 data/FGOALS-f3-L      3219300
 6 data/CMCC-CM2-HR4     3541230
 7 data/MRI-ESM2-0       3037320
 8 data/GFDL-CM4         3219300
 9 data/BCC-CSM2-MR      3035340
10 data/EC-Earth3-Veg-LR 3037320
# ℹ 17 more rows
# ℹ Use `print(n = ...)` to see more rows
Time difference of 0.4325202 secs
CPU times: user 1.23 s, sys: 319 ms, total: 1.55 s
Wall time: 448 ms
